<h1><center>Laboratorio 7: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Matías Rojas y Mauricio Araneda
- Auxiliar: Ignacio Meza D.
- Ayudante: Rodrigo Guerra

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Vanessa Gaete
- Nombre de alumno 2:


### **Link de repositorio de GitHub:** `https://github.com/VanessaGaete/Laboratorio-de-Ciencia-de-Datos`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- Fecha de entrega: TBD
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [43]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [44]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
df_players = pd.read_csv('stats_players.csv')
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [46]:
### Código aquí ###
#1 es para los que califican, 0 para los que no
df_players['label'] = df_players['National_Position'].apply(lambda x: 0 if pd.isna(x) else 1)
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,label
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,95,85,92,93,90,81,76,85,88,1
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,68,71,85,95,88,89,90,74,85,1
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,61,62,78,89,77,79,84,81,83,1
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,69,77,87,94,86,86,84,85,88,1
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,78,25,25,13,16,14,11,47,11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,56,14,16,13,13,11,13,15,12,0
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,53,12,17,12,11,12,13,16,12,0
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,61,41,44,28,42,35,36,42,37,0
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,48,15,23,14,12,13,12,24,12,0


### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [ ] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [ ] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [ ] Crear 3 ``pipelines`` con diferentes clasificadores.
- [ ] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [ ] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

**Respuesta:**

In [47]:
from sklearn.preprocessing import  RobustScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

ct = ColumnTransformer(
    [
        (   "Scaler",
            StandardScaler(), #O con Standard Scaler
            [   "Ball_Control", "Dribbling", "Marking", "Sliding_Tackle","Standing_Tackle", "Interceptions","Crossing", "Short_Pass", "Long_Pass", "Stamina", "Heading", "Shot_Power", "Finishing", "Long_Shots", "Curve", "Freekick_Accuracy", "Penalties", "Volleys",
                "Height",
                "Weight",
                "Aggression",
                "Reactions",
                "Vision",
                "Composure",
                "Acceleration",
                "Speed",
                "Strength",
                "Balance",
                "Agility",
                "Jumping",
                "Weak_foot",
                "Skill_Moves"
            ],),
        ("box-cox",
           PowerTransformer("box-cox"),
          ["Ball_Control", "Age","Dribbling", "Marking", "Sliding_Tackle","Standing_Tackle", "Interceptions","Crossing", "Short_Pass", "Long_Pass", "Stamina", "Heading", "Shot_Power", "Finishing", "Long_Shots", "Curve", "Freekick_Accuracy", "Penalties", "Volleys"],),
        ("OneHot",
            OneHotEncoder(sparse=False, handle_unknown="ignore"),
            ["Nationality", "Club_Position","Preffered_Foot", "Work_Rate"],),
    ]
)


knn_pipe = Pipeline(
    [
        ("preprocesamiento", ct), 
        ("knn", KNeighborsClassifier(n_neighbors=3))
    ]
)



svc_pipe = Pipeline([
    ("preprocesamiento", ct), 
    ("svc", SVC(kernel="linear"))
])

svc_pipe_rbf = Pipeline([
    ("preprocesamiento", ct), 
    ("svc", SVC(kernel="rbf")
    )
])


Se usa un standardScaler sobre todas las caracteristicas, para hacer que los datos se muevan en rangos semejantes. Luego se aplica un Power Transformer sobre las caracteristicas que no tienen una distribución normal o gaussiana, y así con este transformador convertirlas en gaussianas. Finalmente un OneHotEncoder sobre las caracteristicas que son categóricas.


Como clasificadores se utilizará SVM lineal, SVM polinomial y KNN. No se usa Bayes porque este conlleva al supuesto de que no hay variables dependientes, sin embargo, como se puede ver en la matriz de corelación del archivo Player Stats, hay varias variables que están correlacionadas entre sí. Dados los clasificadores a utilizar, tambien se aplica OneHotEncoding a las variables que son caracteristicas.


### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [ ] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar los `pipelines`.
- [ ] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [48]:
### Código aquí ###


# Holdout
from sklearn.model_selection import train_test_split

features = df_players.drop(columns=["Name","National_Position", "label"])
labels = df_players.loc[:, "label"]


display(features)
display(labels)



,Nationality,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,Skill_Moves,Ball_Control,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Portugal,LW,185,80,Right,32,High / Low,4,5,93,...,90,95,85,92,93,90,81,76,85,88
1,Argentina,RW,170,72,Left,29,Medium / Medium,4,4,95,...,90,68,71,85,95,88,89,90,74,85
2,Brazil,LW,174,68,Right,25,High / Medium,5,5,95,...,96,61,62,78,89,77,79,84,81,83
3,Uruguay,ST,182,85,Right,30,High / Medium,4,4,91,...,86,69,77,87,94,86,86,84,85,88
4,Germany,GK,193,92,Right,31,Medium / Medium,4,1,48,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Republic of Ireland,Sub,183,82,Right,19,Medium / Medium,1,1,12,...,27,56,14,16,13,13,11,13,15,12
17584,Republic of Ireland,Sub,185,80,Right,19,Medium / Medium,2,1,13,...,28,53,12,17,12,11,12,13,16,12
17585,England,Res,173,61,Right,18,High / Medium,2,2,44,...,54,61,41,44,28,42,35,36,42,37
17586,Scotland,Sub,180,80,Right,21,Medium / Medium,3,1,17,...,34,48,15,23,14,12,13,12,24,12


0        1
1        1
2        1
3        1
4        1
        ..
17583    0
17584    0
17585    0
17586    0
17587    0
Name: label, Length: 17588, dtype: int64

In [49]:
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.33, shuffle=True, stratify=labels
)

In [50]:
from sklearn.metrics import confusion_matrix

knn_pipe.fit(X_train, y_train)
y_pred = knn_pipe.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_pred, y_test))





[[5386   64]
 [ 326   29]]
              precision    recall  f1-score   support

           0       0.99      0.94      0.97      5712
           1       0.08      0.31      0.13        93

    accuracy                           0.93      5805
   macro avg       0.53      0.63      0.55      5805
weighted avg       0.97      0.93      0.95      5805



Este clasificador no esta prediciendo bien los casos donde el fultbolista sí debiera ser considerado como posible seleccionado nacional, acertando tan solo en un 10% (aprox) de los casos. Se probó con otros valores para los n_neighbours y el valor empeoraba, asi que se dejo en 3.

In [51]:

# Entrenamos
svc_pipe.fit(X_train, y_train)

# Predecimos
y_pred = svc_pipe.predict(X_test)

# Evaluamos
print("Matriz de confusión\n", confusion_matrix(y_pred, y_test))
print(classification_report(y_pred, y_test))



Matriz de confusión
 [[5396  237]
 [  54  118]]
              precision    recall  f1-score   support

           0       0.99      0.96      0.97      5633
           1       0.33      0.69      0.45       172

    accuracy                           0.95      5805
   macro avg       0.66      0.82      0.71      5805
weighted avg       0.97      0.95      0.96      5805



Con este modelo se puede ver que le va mucho mejor al clasificador con los jugadores que sí debiesen ser considerados seleccionados. Sin embargo, aún su precisión en este caso es muy baja, cercano al 30%, obteniendo un mal clasificador de todas formas. 

In [52]:
# Entrenamos
svc_pipe_rbf.fit(X_train, y_train)

# Predecimos
y_pred = svc_pipe_rbf.predict(X_test)

# Evaluamos
print("Matriz de confusión\n", confusion_matrix(y_pred, y_test))
print(classification_report(y_pred, y_test))

Matriz de confusión
 [[5448  353]
 [   2    2]]
              precision    recall  f1-score   support

           0       1.00      0.94      0.97      5801
           1       0.01      0.50      0.01         4

    accuracy                           0.94      5805
   macro avg       0.50      0.72      0.49      5805
weighted avg       1.00      0.94      0.97      5805



Utilizando rbf se obtienen resultados incluso peores que los anteriores. Logrando clasificar bien  a un 0%(aprox) de los valores que debian ser etiquetados seleccionado nacional. Lo que hace este clasificardor, basicamente, es predecir casi todos los datos como no seleccionado nacional, como la cantidad de ejemplos de personas que no son seleccionadas nacionales es mucho mayor al otro caso, se obtienen buena smetricas en promedio pero el desempeño es malo en verdad. 


Con todo lo anterior se consluye que con esta base de datos no es posible predecir correctamente si un futbolista será o no un seleccionado nacional.

## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [ ] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [ ] Cuente cuantos por clase quedan.
- [ ] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [ ] Comente los resultados obtenidos.

**Respuesta:**

In [53]:
### Código aquí ###



def labeling(x):
  if x == "ST" or x == "CF":
    return 0
  elif x == "RW" or x == "CAM" or x == "LW":
    return 1
  elif x == "RM" or x == "CM" or x == "LM":
    return 2
  elif x == "RWB" or x == "CDM" or x == "LWB":
    return 3
  elif x == "RB" or x == "CB" or x == "LB":
    return 4
  elif x == "GK":
    return 5
  else:
    return None


df_position = df_players.copy()
df_position['label'] = df_players['Club_Position'].apply(lambda x: labeling(x))
df_position = df_position.dropna() #Eliminar columnas que no pertenecen a la clasificacion indicada

df_position

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,label
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,95,85,92,93,90,81,76,85,88,1.0
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,68,71,85,95,88,89,90,74,85,1.0
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,61,62,78,89,77,79,84,81,83,1.0
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,69,77,87,94,86,86,84,85,88,0.0
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,78,25,25,13,16,14,11,47,11,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11600,Owain Fôn Williams,Wales,Sub,GK,193,77,Right,30,Medium / Medium,3,...,61,16,13,11,13,11,12,16,11,5.0
11783,Ricardo Reyes,Mexico,LB,LWB,181,66,Left,25,High / High,3,...,62,62,64,34,49,41,46,48,44,3.0
12330,Andraž Struna,Slovenia,RB,RB,185,72,Right,27,Medium / Medium,4,...,74,54,48,32,43,44,47,57,43,4.0
12655,Riley McGree,Australia,Sub,CDM,178,72,Left,18,Medium / Medium,3,...,51,44,50,50,52,49,43,44,40,3.0


In [54]:
df_position["label"].value_counts()

4.0    88
5.0    80
2.0    70
1.0    65
0.0    42
3.0    21
Name: label, dtype: int64

La clase con menos personas es central defensa y la con más es defensa. 

In [55]:

ct = ColumnTransformer(
    [
        (   "Scaler",
            StandardScaler(), #O con Standard Scaler
            [   "Ball_Control", "Dribbling", "Marking", "Sliding_Tackle","Standing_Tackle", "Interceptions","Crossing", "Short_Pass", "Long_Pass", "Stamina", "Heading", "Shot_Power", "Finishing", "Long_Shots", "Curve", "Freekick_Accuracy", "Penalties", "Volleys",
                "Height",
                "Weight",
                "Aggression",
                "Reactions",
                "Vision",
                "Composure",
                "Acceleration",
                "Speed",
                "Strength",
                "Balance",
                "Agility",
                "Jumping",
                "Weak_foot",
                "Skill_Moves"
            ],),
        ("box-cox",
           PowerTransformer("box-cox"),
          ["Ball_Control", "Age","Dribbling", "Marking", "Sliding_Tackle","Standing_Tackle", "Interceptions","Crossing", "Short_Pass", "Long_Pass", "Stamina", "Heading", "Shot_Power", "Finishing", "Long_Shots", "Curve", "Freekick_Accuracy", "Penalties", "Volleys"],),
        ("OneHot",
            OneHotEncoder(sparse=False, handle_unknown="ignore"),
            ["Nationality", "National_Position", "Preffered_Foot", "Work_Rate"],),
    ]
)

knn_pipe2 = Pipeline(
    [
        ("preprocesamiento", ct), 
        ("knn", KNeighborsClassifier(n_neighbors=3))
    ]
)

svc_pipe2 = Pipeline([
    ("preprocesamiento", ct), 
    ("svc", SVC(kernel="linear"))
])

svc_pipe_rbf2 = Pipeline([
    ("preprocesamiento", ct), 
    ("svc", SVC(kernel="rbf")
    )
])


Se vuelven a aplicar le mismo preprocesamiento que en la primera parte, pues los datos a utilizar son los mismos. En cuanto a lo clasificadores, también se utilizan los mismos, pues tenemos la misma cantidad de datos, algunas caracteristicas son dependientes entre si (No s epuede usar Bayes) y no tenemos datos textuales, pues los transformamos.

In [56]:
features = df_position.drop(columns=["Club_Position", "label"])
labels = df_position.loc[:, "label"]

display(features)
display(labels)

X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.33, shuffle=True, stratify=labels
)

,Name,Nationality,National_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,Skill_Moves,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,185,80,Right,32,High / Low,4,5,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,170,72,Left,29,Medium / Medium,4,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,174,68,Right,25,High / Medium,5,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,182,85,Right,30,High / Medium,4,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,193,92,Right,31,Medium / Medium,4,1,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11600,Owain Fôn Williams,Wales,Sub,193,77,Right,30,Medium / Medium,3,1,...,61,61,16,13,11,13,11,12,16,11
11783,Ricardo Reyes,Mexico,LB,181,66,Left,25,High / High,3,2,...,65,62,62,64,34,49,41,46,48,44
12330,Andraž Struna,Slovenia,RB,185,72,Right,27,Medium / Medium,4,2,...,60,74,54,48,32,43,44,47,57,43
12655,Riley McGree,Australia,Sub,178,72,Left,18,Medium / Medium,3,2,...,70,51,44,50,50,52,49,43,44,40


0        1.0
1        1.0
2        1.0
3        0.0
4        5.0
        ... 
11600    5.0
11783    3.0
12330    4.0
12655    3.0
12737    0.0
Name: label, Length: 366, dtype: float64

In [57]:
#ENTRENAMIENTO Y CLASIFICACION KNN
knn_pipe2.fit(X_train, y_train)
y_pred = knn_pipe2.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_pred, y_test))

[[13  1  0  0  0  0]
 [ 5  7  8  0  2  0]
 [ 0  6 13  0  4  0]
 [ 0  4  1  1  1  0]
 [ 0  1  1  0 27  0]
 [ 0  0  0  0  0 26]]
              precision    recall  f1-score   support

         0.0       0.93      0.72      0.81        18
         1.0       0.32      0.37      0.34        19
         2.0       0.57      0.57      0.57        23
         3.0       0.14      1.00      0.25         1
         4.0       0.93      0.79      0.86        34
         5.0       1.00      1.00      1.00        26

    accuracy                           0.72       121
   macro avg       0.65      0.74      0.64       121
weighted avg       0.77      0.72      0.74       121



In [58]:
# Entrenamos
svc_pipe2.fit(X_train, y_train)

# Predecimos
y_pred = svc_pipe2.predict(X_test)

# Evaluamos
print("Matriz de confusión\n", confusion_matrix(y_pred, y_test))
print(classification_report(y_pred, y_test))

Matriz de confusión
 [[13  5  0  0  0  0]
 [ 1  4  8  4  0  0]
 [ 0 12 12  1  0  0]
 [ 0  0  1  1  4  0]
 [ 0  1  2  1 25  0]
 [ 0  0  0  0  0 26]]
              precision    recall  f1-score   support

         0.0       0.93      0.72      0.81        18
         1.0       0.18      0.24      0.21        17
         2.0       0.52      0.48      0.50        25
         3.0       0.14      0.17      0.15         6
         4.0       0.86      0.86      0.86        29
         5.0       1.00      1.00      1.00        26

    accuracy                           0.67       121
   macro avg       0.61      0.58      0.59       121
weighted avg       0.70      0.67      0.68       121



In [59]:
# Entrenamos
svc_pipe_rbf2.fit(X_train, y_train)

# Predecimos
y_pred = svc_pipe_rbf2.predict(X_test)

# Evaluamos
print("Matriz de confusión\n", confusion_matrix(y_pred, y_test))
print(classification_report(y_pred, y_test))

Matriz de confusión
 [[13  3  0  0  0  0]
 [ 0  7  8  2  0  0]
 [ 1 10 12  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  2  3  5 29  0]
 [ 0  0  0  0  0 26]]
              precision    recall  f1-score   support

         0.0       0.93      0.81      0.87        16
         1.0       0.32      0.41      0.36        17
         2.0       0.52      0.52      0.52        23
         3.0       0.00      0.00      0.00         0
         4.0       1.00      0.74      0.85        39
         5.0       1.00      1.00      1.00        26

    accuracy                           0.72       121
   macro avg       0.63      0.58      0.60       121
weighted avg       0.80      0.72      0.75       121



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


De todos los clasificadores, el mejor es el svc rbf, pues obtiene mejores métricas en promedio y de weighted avg también (que corresponde a un promedio donde se considera la diferencia en la cantidad de datos de cada clase). El pero es el lineal. Sin embargo, se puede notar que ningún clasificador es capaz de predecir bien la clase 3. Esto puede deberse a que es la clase con menor cantidad de ejemplos, por lo que se podría mejorar esta situación con un dataset más grande. 


La clasificación de posicion fue mucho mejor lograda que la clasificación de los seleccionados nacionales, sin embargo tampoco se puede decir que fue excelente.


## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [ ] Explique en que consiste la métrica $R^2$
- [ ] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

R¨2 es una metrica que permite ver qué tan buena es la regresión si la coparamos conuna regresión "dummy" que es bastante mala. Para cada punto o elemento del dataset, se calcula el error del valor calculado con el valor real. Si el valor de R^2 es cercano a 1 significa que es un buen modelo y si es negatico es malo, es como haber elegido un  modelo al azar.

In [60]:
df_sueldos = pd.read_csv('sueldos.csv')

df_sueldos.rename(columns={'Player': 'Name'}, inplace=True)
df_sueldos

,Unnamed: 0,Name,Weekly Salary
0,1528,Lionel Messi,1538905.0
1,1006,Cristiano Ronaldo,1248536.0
2,1529,Antoine Griezmann,997032.0
3,2004,Neymar,797726.0
4,1530,Gareth Bale,607706.0
...,...,...,...
2529,1527,Simone Ghidotti,283.0
2530,2539,Valentin Gendrey,100.0
2531,1001,John Yeboah,70.0
2532,1000,Antonis Aidonis,70.0


El dataset resultante del merge tiene menos datos que la tabla de sueldos, porque hay futbolistas en la tabla de sueldos que no estan en el dataset original. Por ejemplo:

In [61]:
df_players["Name"].unique()
assert not "Simone Ghidotti" in df_players["Name"].unique()

In [62]:
merged_df = pd.merge(df_players, df_sueldos, on ='Name', how ="right")
merged_df = merged_df.dropna(subset =["Height"] )
merged_df

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,label,Unnamed: 0,Weekly Salary
0,Lionel Messi,Argentina,RW,RW,170.0,72.0,Left,29.0,Medium / Medium,4.0,...,85.0,95.0,88.0,89.0,90.0,74.0,85.0,1.0,1528,1538905.0
1,Cristiano Ronaldo,Portugal,LS,LW,185.0,80.0,Right,32.0,High / Low,4.0,...,92.0,93.0,90.0,81.0,76.0,85.0,88.0,1.0,1006,1248536.0
2,Antoine Griezmann,France,CAM,RS,176.0,67.0,Left,26.0,High / Medium,3.0,...,81.0,88.0,82.0,84.0,75.0,71.0,87.0,1.0,1529,997032.0
3,Neymar,Brazil,LW,LW,174.0,68.0,Right,25.0,High / Medium,5.0,...,78.0,89.0,77.0,79.0,84.0,81.0,83.0,1.0,2004,797726.0
4,Gareth Bale,Wales,RS,RW,183.0,74.0,Left,27.0,High / Medium,3.0,...,91.0,87.0,90.0,86.0,85.0,76.0,76.0,1.0,1530,607706.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2567,Nicolas Lemaître,France,NaN,Res,185.0,83.0,Right,20.0,Medium / Medium,2.0,...,23.0,17.0,15.0,19.0,14.0,22.0,17.0,0.0,2536,859.0
2569,Andrea Pinamonti,Italy,NaN,Res,181.0,72.0,Right,17.0,Medium / Low,3.0,...,57.0,74.0,60.0,43.0,43.0,69.0,43.0,0.0,1523,805.0
2573,Manu García,Spain,NaN,LDM,182.0,74.0,Left,30.0,Medium / High,3.0,...,74.0,60.0,46.0,52.0,54.0,71.0,42.0,0.0,2538,750.0
2575,Tim Wiesner,Germany,NaN,Res,192.0,87.0,Right,20.0,Medium / Medium,3.0,...,19.0,13.0,15.0,18.0,16.0,20.0,14.0,0.0,989,674.0


Se aplica el mismo preprocesamiento que se aplicó para los casos anteriores, pues se trabaja con los mismos datos, solo que se le añade una columna de sueldos, la cual no se toca con ningún preprocesamiento ya que correpsonde a los targets. 

Dado que la cantidad de datos es menor a 100.000 No se ocupa un SGD Regressor. Luego se checkea si acaso pocas caracteristicas debiesen influir en la decisión. Yo opino que aquí todas o la mayoría de las características del dataset influyen en qué tan bueno o qué tanto rinde realmente un jugador, lo que conlleva evidementemente a una variación en el sueldo. Por lo tanto, según los caminos de la guía, se debe probar con Ridge o SVR linear regressor. Sin embargo, al ver los resultados, se puede ver que ninguno de estos funciona bien, por lo que hay que seguir el camino y llegamos al 
Ensemble Regressor y SVR polinomial, donde finalmente obtenemos que el Ensemble nos da buenos resultados.

In [63]:
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import PowerTransformer


ct = ColumnTransformer(
    [
        (   "Scaler",
            StandardScaler(),
            [   "Ball_Control", "Dribbling", "Marking", "Sliding_Tackle","Standing_Tackle", "Interceptions","Crossing", "Short_Pass", "Long_Pass", "Stamina", "Heading", "Shot_Power", "Finishing", "Long_Shots", "Curve", "Freekick_Accuracy", "Penalties", "Volleys",
                "Height",
                "Weight",
                "Aggression",
                "Reactions",
                "Vision",
                "Composure",
                "Acceleration",
                "Speed",
                "Strength",
                "Balance",
                "Agility",
                "Jumping",
                "Weak_foot",
                "Skill_Moves"
            ],),
        ("box-cox",
           PowerTransformer("box-cox"),
          ["Ball_Control", "Age","Dribbling", "Marking", "Sliding_Tackle","Standing_Tackle", "Interceptions","Crossing", "Short_Pass", "Long_Pass", "Stamina", "Heading", "Shot_Power", "Finishing", "Long_Shots", "Curve", "Freekick_Accuracy", "Penalties", "Volleys"],),
        ("OneHot",
            OneHotEncoder(sparse=False, handle_unknown="ignore"),
            ["Nationality", "National_Position", "Club_Position","Preffered_Foot", "Work_Rate"],),
    ]
)

In [64]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import r2_score

features = merged_df.drop(columns=[ "Name","Weekly Salary", "label", "Unnamed: 0"])
labels = merged_df.loc[:, "Weekly Salary"]

X_train, X_test, y_train, y_test  = train_test_split(
    features, labels, shuffle=True, train_size=0.7, random_state=33
)



In [65]:
from sklearn.svm import SVR
svr_pipe = Pipeline([('preprocesamiento', ct) , 
                       ('svr', SVR(kernel="linear"))])
svr_pipe.fit(X_train, y_train)

svr_y_pred = svr_pipe.predict(X_test)

print('R²:', r2_score(y_test, svr_y_pred))

R²: -0.05241723500087181


In [66]:
rigde_pipe = Pipeline([('preprocesamiento', ct) , 
                       ('regresor', Ridge())])
rigde_pipe.fit(X_train, y_train)

ridge_y_pred = rigde_pipe.predict(X_test)

print('R²:', r2_score(y_test, ridge_y_pred))

R²: 0.22607307841391433


In [67]:
from sklearn.svm import SVR
svr_pipe = Pipeline([('preprocesamiento', ct) , 
                       ('svr', SVR(kernel="rbf"))])
svr_pipe.fit(X_train, y_train)

svr_y_pred = svr_pipe.predict(X_test)

print('R²:', r2_score(y_test, svr_y_pred))

R²: -0.12333117874445887


In [68]:
from sklearn.ensemble import GradientBoostingRegressor
ensemble_pipe = Pipeline(steps=[('preprocessing',ct),
                      ('regressor', GradientBoostingRegressor())])
ensemble_pipe.fit(X_train, y_train)

ensemble_y_pred = ensemble_pipe.predict(X_test)


print('R²:', r2_score(y_test, ensemble_y_pred))

R²: 0.5206549349135501


In [69]:
from sklearn.linear_model import ElasticNet

en_pipe = Pipeline([('preprocesamiento', ct) ,
                    ('regresor', ElasticNet(alpha= 0.09))])
en_pipe.fit(X_train, y_train)

en_y_pred = en_pipe.predict(X_test)


print('R²:', r2_score(y_test, en_y_pred))
#R²: 0.3607911350798436

R²: 0.3607911350798436


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.160e+12, tolerance: 1.021e+09
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


Se puede ver que solo dos regresores son capaces de cumplir con lo pedido. El ensemble regresor y el ElasticNet. El elastic net se probó solo por si acaso, en verdad no correspondía utilizarlo debido al analisis que se hizo anteriormente, aún así es capaz de cumplir con lo solicitado, pero no tan bien como el Ensemble Regressor, lo cual nos dice que el análisis hecho al alegir el regresor estuvo correcto. 

El resultado entregado por el Ensemble Regressor es bastante bueno, es mejor de lo que se esperaba, su valor nos indica que en verdad este regresor si es capaz de predecir correctamente en varios casos, y está lejano a ser un predictor "al azar". Coin esto se concluye que el trabajo estuvo bien logrado. 

La forma en la que se podrían mejorar los resultados es, quizás, estudiando un reductor de la dimensionalidad, que podría mantener las caracteristicas más importantes o influyentes para luego hacer el estudio.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>